## Koopman Model Predictive Control for flow control

In this tutorial we use Model Predictive Control to steer a flow simulation. For this we repeat the results in a similar setting of the paper (Section 5.1) and attached respositpory

* Paper: "A data-driven Koopman model predictive control framework for nonlinear flows" https://arxiv.org/pdf/1804.05291.pdf
* Supplementary code: https://github.com/arbabiha/KoopmanMPC_for_flowcontrol

Note that there are discrepancies between the code and paper description. We mostly align to the setting in the code and highlight discrepancies to the paper.  

The model to be controlled is a one-dimensional Burger equation

$$
\frac{\partial v}{\partial t} + v \frac{\partial v}{\partial x} = \nu \frac{\partial v^2}{\partial^2 x} + f(x,t)
$$

with 

$$
x \in [0, 2\pi] \\
t \in [0, \infty]
$$

and periodic boundary conditions $v(0, t) = v(2\pi, t)$.

*In the paper the domain is $x \in [0, 1]$, while the our choice matches the setting in the supplementary code.*

There are two control parameters $u=(u_1, u_2)$ with contrains $-0.1 \leq u_{1,2} \leq 0.1$. With these we can steer the system state to a reference time series. The control parameters affect the state with:

$$
\begin{align}
f(x,t) &= u_1(t) f_1(x) + u_2(t)f_2(x)\\
&= u_1(t) \exp(-((15/(2\pi) (x - \pi/2))^2)) + u_2(t) \exp(-((15/(2\pi) (x - 3/2\pi))^2)))
\end{align}
$$

*The function parameters are adapted to the larger domain $x \in [0, 2\pi]$ in contrast to the paper.*

We specify an initial condition of the system with a functional relation

$$
v(x, 0) = a \exp(-(((x - \pi) 5/(2\pi))^2)) + (1 - a) \sin(2x)^2
$$

where we uniformly sample $a \in (0,1)$.

*(The function parameters are adapted to the larger domain $x \in [0, 2\pi]$ in contrast to the paper.)*

The control objective is to follow the reference state

$$
\begin{align}
v_{ref} (x, 0 \leq t \leq 20) &= 0.5\\
v_{ref} (x, 20 \leq t \leq 40) &= 1\\
v_{ref} (x, 40 \leq t \leq 60) &= 0.5
\end{align}
$$

*In the paper the time frequency is stated as 0.01 to integrate the system. However, in the final data collection in the code only every 19th state is actually taken, leading to to an effective time interval of $\Delta t = 0.19$. Therefore, the time intervals in $v_{ref}$ in the reference are adapted such that they roughly match the setting in the paper.*

To obtain a suitable control sequence $u$ to steer an initial condition towards $v_{ref}$, we use Koopman-based Model Predictive Control (KMPC). After we sample example data, we identify the system dynamics with the Extended Dynamic Mode Decomposition (EDMD).

In [1]:
import matplotlib.pyplot as plt

from IPython.display import HTML
import numpy as np
import pandas as pd
from matplotlib.animation import FuncAnimation
from scipy.interpolate import interp1d
from sklearn.base import BaseEstimator
from tqdm import tqdm
from datafold import (
    EDMD,
    DMDControl,
    TSCColumnTransformer,
    TSCDataFrame,
    TSCIdentity,
    TSCTakensEmbedding,
    TSCTransformerMixin,
)
from datafold.appfold.mpc import LinearKMPC
from datafold.utils._systems import Burger1DPeriodicBoundary

In [2]:
rng = np.random.default_rng(2)

### Data collection from the original system

Set up Burger system, control functions (f1 and f2) and the initial condition function. We sample the full Burger system, but for EDMD we only use every 10th spatial point in the domain.

In [3]:
sys = Burger1DPeriodicBoundary(n_spatial_points=100, nu=0.01)

# control function
f1 = np.atleast_2d(np.exp(-((15/(2*np.pi) * (sys.x_nodes - 0.5*np.pi)) ** 2)))
f2 = np.atleast_2d(np.exp(-((15/(2*np.pi) * (sys.x_nodes - 1.5*np.pi)) ** 2)))

# initial condition function
ic1 = np.exp(-(((sys.x_nodes - 2*np.pi*0.5) * 5/(2*np.pi)) ** 2))
ic2 = np.sin(2 * sys.x_nodes) ** 2
icfunc = lambda a: a * ic1 + (1 - a) * ic2

In [4]:
# sampling options
dt = 0.19  # time frequency
sim_length = 200 # time series length 
training_size = 100 # number of initial conditions to sample

# function to subselect states to every 10th spatial point
def subselect_measurements(tscdf):
    return tscdf.iloc[:, 9::10]

# contrains on the effective control parameters
umin, umax = (-0.1, 0.1)

# time values of a single time series
time_values = np.arange(0, dt * sim_length + 1e-12, dt)

#### Exectue system sampling

Fill lists of time series by resetting the the initial condition with $a \sim \operatorname{Uniform(0,1)}$. The final data is then captured in a single `TSCDataFrame` as *datafold*'s main data structure.

In [5]:

# lists to collect both time series and control 
X_tsc = []  
U_tsc = []

for i in tqdm(range(training_size)):
    # sample a new initial condition
    ic = icfunc(rng.uniform(0, 1))   
    
    # sample random control parameters (for each timestep) over the simulation horizon
    # describe the control input as a function f(t, x)
    rand_vals = rng.uniform(umin, umax, size=(len(time_values), 2))
    U1rand = lambda t: np.atleast_2d(interp1d(time_values, rand_vals[:, 0], kind="previous")(t)).T
    U2rand = lambda t: np.atleast_2d(interp1d(time_values, rand_vals[:, 1], kind="previous")(t)).T
        
    def f(t, x):
        # while we have 2 effective control inputs they affect each state coordinate according to this function 
        return U1rand(t) * f1 + U2rand(t) * f2
    
    # perform system prediction
    X_predict, Ufull = sys.predict(
        ic, U=f, time_values=time_values, require_last_control_state=False
    )

    # drop last control input, because for the last state no prediction is performed
    U = TSCDataFrame.from_array(
        rand_vals[:-1, :],
        time_values=Ufull.time_values(),
        feature_names=["u1", "u2"],
    )

    X_tsc.append(X_predict)
    U_tsc.append(U)

# finalize the time series collection data to be used for system identification with control input
X_tsc = TSCDataFrame.from_frame_list(X_tsc)
U_tsc = TSCDataFrame.from_frame_list(U_tsc)

100%|█████████████████████████████████████████| 100/100 [00:33<00:00,  2.94it/s]


Snapshot of sampled system states and control input.

In [6]:
print(X_tsc.n_timeseries)
X_tsc

100


feature         x0        x1        x2        x3        x4        x5  \
ID time                                                                
0  0.00   0.000505  0.012482  0.047402  0.103049  0.175877  0.261248   
   0.19   0.006719  0.020279  0.049590  0.093109  0.147763  0.210579   
   0.38   0.013534  0.025553  0.050389  0.085817  0.129411  0.179012   
   0.57   0.021277  0.029966  0.050732  0.080297  0.116383  0.157253   
   0.76   0.032504  0.034800  0.051237  0.076090  0.106645  0.141263   
...            ...       ...       ...       ...       ...       ...   
99 37.24  0.263713  0.266042  0.268830  0.271891  0.275108  0.278402   
   37.43  0.262967  0.264520  0.266806  0.269544  0.272556  0.275722   
   37.62  0.263397  0.263788  0.265309  0.267551  0.270242  0.273203   
   37.81  0.265509  0.264208  0.264588  0.266078  0.268278  0.270924   
   38.00  0.269947  0.266279  0.265003  0.265373  0.266831  0.268991   

feature         x6        x7        x8        x9  ...       x90       x91  \
ID time                                           ...                       
0  0.00   0.353723  0.447418  0.536375  0.614951  ...  0.614951  0.536375   
   0.19   0.278910  0.350355  0.422632  0.493428  ...  0.714647  0.666874   
   0.38   0.232879  0.289600  0.347978  0.406918  ...  0.720190  0.725388   
   0.57   0.201584  0.248367  0.296807  0.346237  ...  0.662216  0.692435   
   0.76   0.178820  0.218506  0.259708  0.301943  ...  0.592642  0.628560   
...            ...       ...       ...       ...  ...       ...       ...   
99 37.24  0.281722  0.285035  0.288321  0.291574  ...  0.365354  0.346123   
   37.43  0.278968  0.282243  0.285518  0.288780  ...  0.379645  0.364582   
   37.62  0.276320  0.279515  0.282740  0.285964  ...  0.390106  0.379266   
   37.81  0.273839  0.276910  0.280065  0.283259  ...  0.397784  0.390146   
   38.00  0.271591  0.274460  0.277486  0.280599  ...  0.403939  0.398239   

feature        x92       x93       x94       x95       x96       x97  \
ID time                                                                
0  0.00   0.447418  0.353723  0.261248  0.175877  0.103049  0.047402   
   0.19   0.588267  0.481991  0.360134  0.240733  0.140171  0.067735   
   0.38   0.705878  0.646697  0.530618  0.369256  0.213341  0.103245   
   0.57   0.712929  0.719328  0.696787  0.590587  0.380482  0.183554   
   0.76   0.661008  0.687498  0.708114  0.726475  0.652629  0.392981   
...            ...       ...       ...       ...       ...       ...   
99 37.24  0.324603  0.304028  0.287056  0.274830  0.267140  0.263102   
   37.43  0.345277  0.323968  0.303788  0.287235  0.275339  0.267857   
   37.62  0.363897  0.344530  0.323423  0.303610  0.287444  0.275853   
   37.81  0.378897  0.363246  0.343851  0.322970  0.303514  0.287700   
   38.00  0.390232  0.378627  0.362740  0.343302  0.322584  0.303431   

feature        x98       x99  
ID time                       
0  0.00   0.012482  0.000505  
   0.19   0.025047  0.007445  
   0.38   0.042452  0.016947  
   0.57   0.074708  0.031057  
   0.76   0.160064  0.059943  
...            ...       ...  
99 37.24  0.261713  0.262126  
   37.43  0.263922  0.262566  
   37.62  0.268563  0.264725  
   37.81  0.276380  0.269261  
   38.00  0.287949  0.276895  

[20100 rows x 100 columns]

In [7]:
U_tsc

feature         u1        u2
ID time                     
0  0.00  -0.040302  0.062845
   0.19  -0.081617  0.020020
   0.38   0.045712 -0.062420
   0.57  -0.088971 -0.045006
   0.76   0.031487  0.012453
...            ...       ...
99 37.05  0.099268 -0.032256
   37.24  0.082136 -0.021325
   37.43 -0.081171  0.048061
   37.62  0.075123 -0.045764
   37.81 -0.003650  0.075037

[20000 rows x 2 columns]

Animate sampled time series data with control input. 

In [8]:
tsid = 0  # select time series ID to plot

f, ax = plt.subplots(figsize=(8, 7), nrows=2)
plt.close()  # close to perform video animation, the figure still exists

(ref_line,) = ax[0].plot(sys.x_nodes, X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy(), label="model")
ax[0].legend(loc="upper left")

def Ufunc(u):
    return u[0] * f1 + u[1] * f2

(control_line,) = ax[1].plot(sys.x_nodes,
                       Ufunc(U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy()).ravel(),
                       label="randomly sampled control input")
ax[1].set_ylim(-0.1, 0.1)
ax[1].legend(loc="upper left")

def func(i):
    ref_line.set_ydata(X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy())
    vals = U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy()
    control_line.set_ydata(Ufunc(vals))
    return (ref_line, control_line, )

anim = FuncAnimation(f, func=func, frames=U_tsc.n_timesteps);
HTML(anim.to_html5_video())

## Set up system identification and control input using Extended Dynamic Mode Decomposition 

For the system identification we adapt the sampled data as follows:

* We only use a reduced number of spatial points (every 10th grid point), which reduces the dimensionality of the system.
* We attach the control input in $U$ to the system states in $X$. For this, we also shift the time index in $U$ by one such that the (past) control input is attached to the actual resulting system state. Because of this time shift the initial system state has no matching control input. We fill this with zeros, however, these values are ignored later when performing a time delay embedding in the EDMD dictionary.

**Note:**

In EDMD the system states and control input are treated separately. This means attaching the control input to the system states is only done to enrich the system state and not a mandatory step when using EDMD with control.  

In [9]:
def shift_time_index_U(_X, _U):
    new_index = _X.groupby("ID").tail(_X.n_timesteps - 1).index
    return _U.set_index(new_index)

In [10]:
# use only selected spatial points
X_tsc_reduced = subselect_measurements(X_tsc)

# attach control input to system state
X_tsc_reduced = pd.concat(
    [X_tsc_reduced, shift_time_index_U(X_tsc_reduced, U_tsc)], axis=1
)

# fill nan values with 0 where no corresponding control input was available
X_tsc_reduced = X_tsc_reduced.fillna(0) 

Display effective system states used within EDMD

In [11]:
X_tsc_reduced

feature         x9       x19       x29       x39       x49       x59  \
ID time                                                                
0  0.00   0.614951  0.352659  0.284394  0.894827  0.264416  0.852158   
   0.19   0.493428  0.471498  0.224810  0.888657  0.293167  0.706928   
   0.38   0.406918  0.629393  0.183127  0.784118  0.357912  0.590388   
   0.57   0.346237  0.717336  0.166435  0.673454  0.530439  0.507621   
   0.76   0.301943  0.684019  0.141133  0.583136  0.838769  0.448508   
...            ...       ...       ...       ...       ...       ...   
99 37.24  0.291574  0.338177  0.425919  0.375045  0.370907  0.396930   
   37.43  0.288780  0.335574  0.430290  0.382805  0.368453  0.393959   
   37.62  0.285964  0.325188  0.417955  0.391581  0.366451  0.390983   
   37.81  0.283259  0.323683  0.414728  0.401027  0.365069  0.387976   
   38.00  0.280599  0.318689  0.406318  0.410129  0.364503  0.385053   

feature        x69       x79       x89       x99        u1        u2  
ID time                                                               
0  0.00   0.381333  0.265853  0.678180  0.000505  0.000000  0.000000  
   0.19   0.551812  0.225140  0.734218  0.007445 -0.040302  0.062845  
   0.38   0.805753  0.197675  0.698197  0.016947 -0.081617  0.020020  
   0.57   0.885717  0.170645  0.625450  0.031057  0.045712 -0.062420  
   0.76   0.832479  0.151501  0.554260  0.059943 -0.088971 -0.045006  
...            ...       ...       ...       ...       ...       ...  
99 37.24  0.414674  0.427681  0.380107  0.262126  0.099268 -0.032256  
   37.43  0.411750  0.426739  0.390062  0.262566  0.082136 -0.021325  
   37.62  0.412523  0.429550  0.397360  0.264725 -0.081171  0.048061  
   37.81  0.408690  0.425468  0.403187  0.269261  0.075123 -0.045764  
   38.00  0.412067  0.429602  0.408500  0.276895 -0.003650  0.075037  

[20100 rows x 12 columns]

### Setting up EDMD with dictionary and underlying DMD model

We now set up the EDMD dictionary (as a pipeline). First we specify a custom data transformation to compute the L2 norm from the sates (as described in paper).

In [12]:
class L2Norm(BaseEstimator, TSCTransformerMixin):
    def fit(self, X):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["L2norm"]

    def transform(self, X: TSCDataFrame, y=None):
        return TSCDataFrame.from_same_indices_as(
            X,
            np.sum(np.square(np.abs(X.to_numpy())), axis=1) / X.shape[1],
            except_columns=self.get_feature_names_out(),
        )

Now we describe the dictionary, where we compute the L2-norm on the system states, time delay embedding and also add a constant vector.  

In [13]:
l2norm = ("l2_x", L2Norm(), lambda df: df.columns.str.startswith("x"))

delay1 = (
    "delay_x",
    TSCTakensEmbedding(delays=4),
    lambda df: df.columns.str.startswith("x"),
)
delay2 = (
    "delay_u",
    TSCTakensEmbedding(delays=3),
    lambda df: df.columns.str.startswith("u"),
)

_dict = (
    "tde",
    TSCColumnTransformer([l2norm, delay1, delay2], verbose_feature_names_out=False),
)

_id = ("_id", TSCIdentity(include_const=True))

dict_steps = [_dict, _id]

# It is essential to use DMD with control in EDMD. 
# The flag `dict_preserves_id_states=True` indicates that the original states are contained in the dictionary,
# which makes the inverse mapping from dictionary states to full states easier as it is only a projection.
edmd = EDMD(dict_steps, dmd_model=DMDControl(), include_id_state=False, dict_preserves_id_state=True)

# display html representation of object
edmd

EDMD(dict_preserves_id_state=True,
     dict_steps=[('tde',
                  TSCColumnTransformer(transformers=[('l2_x', L2Norm(),
                                                      <function <lambda> at 0x7f6e14e27430>),
                                                     ('delay_x',
                                                      TSCTakensEmbedding(delays=4),
                                                      <function <lambda> at 0x7f6e14e274c0>),
                                                     ('delay_u',
                                                      TSCTakensEmbedding(delays=3),
                                                      <function <lambda> at 0x7f6e14e27790>)],
                                       verbose_feature_names_out=False)),
                 ('_id', TSCIdentity(include_const=True))],
     dmd_model=DMDControl(), include_id_state=False)

With the specified EDMD model, we can now fit the model with the sampled data comprising states `X` and control input `U`.

In [14]:
edmd.fit(X_tsc_reduced, U=U_tsc);

We can now look at the time series data in the dictionary space. Note that the first samples of the time series are dropped due to the time delay embedding. The number of samples in `X` necessary to map from full-state to dictionary state is available in the attribute `edmd.n_samples_ic_`. 

In [15]:
print(f"{edmd.n_samples_ic_=}")
edmd.transform(X_tsc_reduced)

5


feature     L2norm        x9       x19       x29       x39       x49  \
ID time                                                                
0  0.76   0.285048  0.301943  0.684019  0.141133  0.583136  0.838769   
   0.95   0.254096  0.268404  0.632522  0.133444  0.512709  0.873360   
   1.14   0.224823  0.242066  0.571858  0.119214  0.457050  0.794670   
   1.33   0.231303  0.220925  0.525406  0.124385  0.412456  0.731506   
   1.52   0.226133  0.203548  0.484013  0.156189  0.376019  0.669866   
...            ...       ...       ...       ...       ...       ...   
99 37.24  0.138463  0.291574  0.338177  0.425919  0.375045  0.370907   
   37.43  0.139139  0.288780  0.335574  0.430290  0.382805  0.368453   
   37.62  0.138536  0.285964  0.325188  0.417955  0.391581  0.366451   
   37.81  0.138474  0.283259  0.323683  0.414728  0.401027  0.365069   
   38.00  0.139263  0.280599  0.318689  0.406318  0.410129  0.364503   

feature        x59       x69       x79       x89  ...    x99:d4        u1  \
ID time                                           ...                       
0  0.76   0.448508  0.832479  0.151501  0.554260  ...  0.000505 -0.088971   
   0.95   0.405392  0.757625  0.143187  0.494160  ...  0.007445  0.031487   
   1.14   0.374325  0.684959  0.141168  0.444787  ...  0.016947 -0.069988   
   1.33   0.357196  0.624040  0.190619  0.404132  ...  0.031057  0.033859   
   1.52   0.381356  0.581099  0.541359  0.370507  ...  0.059943  0.026637   
...            ...       ...       ...       ...  ...       ...       ...   
99 37.24  0.396930  0.414674  0.427681  0.380107  ...  0.269070  0.099268   
   37.43  0.393959  0.411750  0.426739  0.390062  ...  0.266582  0.082136   
   37.62  0.390983  0.412523  0.429550  0.397360  ...  0.264460 -0.081171   
   37.81  0.387976  0.408690  0.425468  0.403187  ...  0.262885  0.075123   
   38.00  0.385053  0.412067  0.429602  0.408500  ...  0.262126 -0.003650   

feature         u2     u1:d1     u2:d1     u1:d2     u2:d2     u1:d3  \
ID time                                                                
0  0.76  -0.045006  0.045712 -0.062420 -0.081617  0.020020 -0.040302   
   0.95   0.012453 -0.088971 -0.045006  0.045712 -0.062420 -0.081617   
   1.14  -0.013474  0.031487  0.012453 -0.088971 -0.045006  0.045712   
   1.33  -0.015443 -0.069988 -0.013474  0.031487  0.012453 -0.088971   
   1.52   0.093487  0.033859 -0.015443 -0.069988 -0.013474  0.031487   
...            ...       ...       ...       ...       ...       ...   
99 37.24 -0.032256  0.042654  0.054356  0.092497  0.007993  0.032277   
   37.43 -0.021325  0.099268 -0.032256  0.042654  0.054356  0.092497   
   37.62  0.048061  0.082136 -0.021325  0.099268 -0.032256  0.042654   
   37.81 -0.045764 -0.081171  0.048061  0.082136 -0.021325  0.099268   
   38.00  0.075037  0.075123 -0.045764 -0.081171  0.048061  0.082136   

feature      u2:d3  const  
ID time                    
0  0.76   0.062845      1  
   0.95   0.020020      1  
   1.14  -0.062420      1  
   1.33  -0.045006      1  
   1.52   0.012453      1  
...            ...    ...  
99 37.24  0.017861      1  
   37.43  0.007993      1  
   37.62  0.054356      1  
   37.81 -0.032256      1  
   38.00 -0.021325      1  

[19700 rows x 60 columns]

## Set up control optimization with Model Predictive Control 

Prediction parameters:

In [16]:
horizon = 20  # the horizon is the number of steps
Tpred = dt * horizon  # prediction horizon in MPC
Tend = 70  # end time to predict time series
Nsim = int(Tend // dt) + 1  # number of simulation steps in MPC loop

Because we need 5 states to map to a dictionary space, we first perform a warm-up phase where we evaluate the system with no control applied.

In [17]:
ic = icfunc(0.2)  # can also be adapted to control another time series

X_init, _ = sys.predict(
    ic,
    # Why edmd.n_samples_ic_-1 in U:
    # -> typically no control input is needed for the final system state
    U=np.zeros((edmd.n_samples_ic_-1, sys.n_control_in_)),  
    time_values=np.arange(0, edmd.n_samples_ic_ * dt, dt),
)

Set up `LinearKMPC` model which will optimize for control sequence.  

In [18]:
kmpc = LinearKMPC(
    edmd=edmd,
    horizon=horizon,
    input_bounds=np.array([[-0.1, 0.1], [-0.1, 0.1]]),
    state_bounds=None, 
    qois=X_tsc_reduced.columns[X_tsc_reduced.columns.str.startswith("x")],
    cost_running=1,
    cost_terminal=1,
    cost_input=1,
)

Generate the reference time series (both in full and reduced coordinates).

In [19]:
start_time = X_init.time_values()[-1]
time_values_ref = np.arange(0, start_time + Tend, dt)

X_ref = np.zeros(len(time_values_ref))
X_ref[time_values_ref <= 20] = 0.5
X_ref[np.logical_and(time_values_ref > 20, time_values_ref < 40)] = 1
X_ref[time_values_ref > 40] = 0.5
X_ref = np.outer(X_ref, np.ones(X_tsc.shape[1]))
X_ref = TSCDataFrame.from_array(
    X_ref, time_values=time_values_ref, feature_names=X_tsc.columns
)

X_ref_reduced = subselect_measurements(X_ref)

U_ic = TSCDataFrame.from_array(
    np.zeros((edmd.n_samples_ic_-1, 2)), time_values=X_init.time_values()[:-1], feature_names=edmd.control_names_in_
)

X_ref_reduced

feature    x9  x19  x29  x39  x49  x59  x69  x79  x89  x99
ID time                                                   
0  0.00   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.19   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.38   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.57   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.76   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
   69.92  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   70.11  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   70.30  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   70.49  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   70.68  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5

[373 rows x 10 columns]

In [20]:
# keep track of current system and EDMD state
model_state = X_init.iloc[[-1], :]
edmd_state = pd.concat([subselect_measurements(X_init), shift_time_index_U(X_init, U_ic)], axis=1).fillna(0)

# recotrd the model evolution and optimized control input
X_model_evolution = X_init
U_evolution = U_ic

# record the uncontrolled time series for comparison 
X_model_unctr_evolution = X_init.copy()

# record system error between state and reference time series
X_error_evolution = X_init - X_ref.iloc[[0], :].to_numpy()

for i in tqdm(range(X_init.shape[0], Nsim)):
    
    # reference time series over horizon
    reference = X_ref_reduced.iloc[i: i + horizon, :]
    
    if reference.shape[0] != kmpc.horizon:
        # stop loop if the rest of reference signal is smaller than horizon
        break
    
    t = X_model_evolution.time_values()[-1]
    t_new = X_model_evolution.time_values()[-1] + dt
    
    # optimize the control input on EDMD
    U = kmpc.control_sequence(edmd_state, reference=reference)

    # use only the first control input for the next step
    Ufull = U.iloc[0, 0] * f1 + U.iloc[0, 1] * f2
    U_evolution = pd.concat([U_evolution, U.iloc[[0], :]], axis=0)
    
    # apply the the obtained control input from the EDMD to the actual system
    X_model, _ = sys.predict(X_model_evolution.iloc[[-1], :], U=Ufull, time_values=np.array([t, t_new]))
    X_model_evolution = pd.concat([X_model_evolution, X_model.iloc[[1], :]], axis=0)
    
    # record difference between model and reference 
    diff = X_model.iloc[[1], :] - X_ref.iloc[[i], :].to_numpy()
    X_error_evolution = pd.concat([X_error_evolution, diff])
    
    # perform separate uncontrolled system 
    X_model_unctr, _ = sys.predict(X_model_unctr_evolution.iloc[[-1], :], U=np.zeros_like(sys.x_nodes)[np.newaxis, :], time_values=np.array([t, t_new]))
    X_model_unctr_evolution = pd.concat([X_model_unctr_evolution, X_model_unctr.iloc[[1], :]], axis=0)

    # prepare new edmd_state for next iteration (attach the shifted control input)
    X_model_last = subselect_measurements(X_model_evolution.iloc[-edmd.n_samples_ic_ :, :])
    U_last = U_evolution.iloc[-edmd.n_samples_ic_ :-1, :]
    U_last_shifted = shift_time_index_U(X_model_last, shift_time_index_U(X_model_last, U_last))
    edmd_state = pd.concat([X_model_last, U_last_shifted], axis=1).fillna(0)

 96%|███████████████████████████████████████▎ | 349/364 [01:01<00:02,  5.69it/s]


Animate the controlled and uncontrolled system. Note that because of the state prediction, the control input already changes before `t=20`, where the reference state is changed from $v_{ref}=0.5$ to $v_{ref}=1$. 

In [21]:
f, ax = plt.subplots(figsize=(8, 8), nrows=3, sharex=True)
plt.close() # see https://stackoverflow.com/a/47138474 (first comment)
(model_line,) = ax[0].plot(sys.x_nodes, X_model_evolution.iloc[0], label="system controlled")
(model_uctr_line,) = ax[0].plot(sys.x_nodes, X_model_unctr_evolution.iloc[0], label="system uncontrolled")
(ref_line,) = ax[0].plot(sys.x_nodes, X_ref.iloc[0], label="reference")
ax[0].legend(loc="upper left")
ax[0].set_ylim(0, 1.3)

Ufunc = lambda u, x: (u[0] * f1 + u[1] * f2).ravel()
(control_line,) = ax[1].plot(
    sys.x_nodes,
    Ufunc(U_evolution.iloc[0, :].to_numpy(), None),
    label="control",
)
ax[1].set_ylim(umin, umax)
ax[1].legend(loc="upper left")

(error_line,) = ax[2].plot(sys.x_nodes, X_error_evolution.iloc[0, :].to_numpy(), c="red", label="difference")
ax[2].legend(loc="upper left")

def func(i):
    model_line.set_ydata(X_model_evolution.iloc[i, :].to_numpy())
    model_uctr_line.set_ydata(X_model_unctr_evolution.iloc[i, :].to_numpy())
    ref_line.set_ydata(X_ref.iloc[i, :].to_numpy())
    control_line.set_ydata(Ufunc(U_evolution.iloc[i].to_numpy(), None))
    error_line.set_ydata(X_error_evolution.iloc[i, :].to_numpy())

    return (
        model_line,
        model_uctr_line,
        ref_line,
        error_line
    )

anim = FuncAnimation(f, func=func, frames=U_evolution.shape[0]);
HTML(anim.to_html5_video())